In [3]:
from open_flamingo import create_model_and_transforms
from omegaconf import OmegaConfd
import torch

In [5]:
cfg = dict(
    model_family = 'kosmos',
    lm_path = 'microsoft/Phi-3-mini-4k-instruct',
    # vision_encoder_path = 'ViT-H-14-378-quickgelu',
    # vision_encoder_pretrained = 'dfn5b',
    vision_encoder_path = 'google/siglip-so400m-patch14-384',
    vision_encoder_pretrained = 'google',
    num_vision_tokens = 128,
    image_aspect_ratio = 'anyres',
    anyres_patch_sampling = True,
    anyres_grids=[[1,2],[2,1],[2,2],[3,1],[1,3]],
    ckpt_pth = '/export/share/manli_shu/models/open-flamingo-dev/anyres_ablation_HFSiglip_patch128-kosmos_non_instruct-phi3_4k_instruct_nq128_pre_V3_5-llava_1p6_ocrmathmix_v4-8x8-ckpt2/checkpoint_0.pt',
)
cfg = OmegaConf.create(cfg)
if cfg.model_family in ['kosmos-instruct', 'kosmos', 'llava']:
    additional_kwargs = {
        "image_aspect_ratio": cfg.image_aspect_ratio,
        }
    if cfg.model_family in ['kosmos-instruct', 'kosmos']:
        additional_kwargs.update({
            "num_vision_tokens": cfg.num_vision_tokens,
            "anyres_patch_sampling": cfg.anyres_patch_sampling,
        })
print(additional_kwargs)

{'image_aspect_ratio': 'anyres', 'num_vision_tokens': 128, 'anyres_patch_sampling': True}


In [46]:
model, image_processor, tokenizer = create_model_and_transforms(
    clip_vision_encoder_path=cfg.vision_encoder_path,
    clip_vision_encoder_pretrained=cfg.vision_encoder_pretrained,
    lang_model_path=cfg.lm_path,
    tokenizer_path=cfg.lm_path,
    model_family=cfg.model_family,
    **additional_kwargs)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
`flash-attention` package not found, consider installing for better performance: No module named 'flash_attn'.
Current `flash-attention` does not support `window_size`. Either upgrade or use `attn_implementation='eager'`.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

kosmos model initialized with 3,931,031,619 trainable parameters
==========Trainable Parameters
Vision encoder: 0 trainable parameters
Vision tokenizer: 109,901,568 trainable parameters
Language model: 3,821,130,051 trainable parameters
==========Total Parameters
Vision encoder: 428,225,600 parameters
Vision tokenizer: 109,901,568 parameters
Language model: 3,821,130,051 parameters


In [47]:
ckpt = torch.load(cfg.ckpt_pth)["model_state_dict"]

In [51]:
model.vision_tokenizer

PerceiverResampler(
  (projection): Linear(in_features=1152, out_features=3072, bias=True)
  (layers): ModuleList(
    (0-5): 6 x ModuleList(
      (0): PerceiverAttention(
        (norm_media): LayerNorm((1152,), eps=1e-05, elementwise_affine=True)
        (norm_latents): LayerNorm((1152,), eps=1e-05, elementwise_affine=True)
        (to_q): Linear(in_features=1152, out_features=1536, bias=False)
        (to_kv): Linear(in_features=1152, out_features=3072, bias=False)
        (to_out): Linear(in_features=1536, out_features=1152, bias=False)
      )
      (1): Sequential(
        (0): LayerNorm((1152,), eps=1e-05, elementwise_affine=True)
        (1): Linear(in_features=1152, out_features=4608, bias=False)
        (2): GELU(approximate='none')
        (3): Linear(in_features=4608, out_features=1152, bias=False)
      )
    )
  )
  (norm): LayerNorm((1152,), eps=1e-05, elementwise_affine=True)
)

In [55]:
image_processor

Compose(
    Resize(size=(384, 384), interpolation=bicubic, max_size=None, antialias=True)
    Lambda()
    ToTensor()
    Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
)

In [39]:
model.vision_encoder.config.save_pretrained('/export/share/yutong/xgenmm/llamacpp_wd/siglip_kosmos_phi3_4k_instruct/vit_config')

In [25]:
image_processor

Compose(
    Resize(size=(384, 384), interpolation=bicubic, max_size=None, antialias=True)
    Lambda()
    ToTensor()
    Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
)

In [3]:
from transformers.models.siglip.modeling_siglip import SiglipVisionTransformer, SiglipVisionConfig
import json
import torch
with open('/export/share/yutong/xgenmm/llamacpp_wd/siglip_kosmos_phi3_4k_instruct/vision_encoder/config.json', 'r') as f:
    vision_config = json.load(f)
vision_config = SiglipVisionConfig(**vision_config)
vision_encoder = SiglipVisionTransformer(vision_config)
vit_ckpt = torch.load('/export/share/yutong/xgenmm/llamacpp_wd/siglip_kosmos_phi3_4k_instruct/vision_encoder/xgenmm.vision_encoder')
vision_encoder.load_state_dict(vit_ckpt, strict=True)

<All keys matched successfully>

In [5]:
state_dict = vision_encoder.state_dict()

In [6]:
state_dict.keys()

odict_keys(['embeddings.patch_embedding.weight', 'embeddings.patch_embedding.bias', 'embeddings.position_embedding.weight', 'encoder.layers.0.self_attn.k_proj.weight', 'encoder.layers.0.self_attn.k_proj.bias', 'encoder.layers.0.self_attn.v_proj.weight', 'encoder.layers.0.self_attn.v_proj.bias', 'encoder.layers.0.self_attn.q_proj.weight', 'encoder.layers.0.self_attn.q_proj.bias', 'encoder.layers.0.self_attn.out_proj.weight', 'encoder.layers.0.self_attn.out_proj.bias', 'encoder.layers.0.layer_norm1.weight', 'encoder.layers.0.layer_norm1.bias', 'encoder.layers.0.mlp.fc1.weight', 'encoder.layers.0.mlp.fc1.bias', 'encoder.layers.0.mlp.fc2.weight', 'encoder.layers.0.mlp.fc2.bias', 'encoder.layers.0.layer_norm2.weight', 'encoder.layers.0.layer_norm2.bias', 'encoder.layers.1.self_attn.k_proj.weight', 'encoder.layers.1.self_attn.k_proj.bias', 'encoder.layers.1.self_attn.v_proj.weight', 'encoder.layers.1.self_attn.v_proj.bias', 'encoder.layers.1.self_attn.q_proj.weight', 'encoder.layers.1.self_a

In [8]:
state_dict['embeddings.position_embedding.weight'].shape

torch.Size([729, 1152])

In [9]:
state_dict['embeddings.position_embedding.weight'].unsqueeze(0).shape

torch.Size([1, 729, 1152])

In [10]:
projector = torch.load('/export/share/yutong/xgenmm/llamacpp_wd/siglip_kosmos_phi3_4k_instruct/xgenmm.projector')

In [11]:
for k in projector.keys():
    print(k)

latents
projection.weight
projection.bias
layers.0.0.norm_media.weight
layers.0.0.norm_media.bias
layers.0.0.norm_latents.weight
layers.0.0.norm_latents.bias
layers.0.0.to_q.weight
layers.0.0.to_kv.weight
layers.0.0.to_out.weight
layers.0.1.0.weight
layers.0.1.0.bias
layers.0.1.1.weight
layers.0.1.3.weight
layers.1.0.norm_media.weight
layers.1.0.norm_media.bias
layers.1.0.norm_latents.weight
layers.1.0.norm_latents.bias
layers.1.0.to_q.weight
layers.1.0.to_kv.weight
layers.1.0.to_out.weight
layers.1.1.0.weight
layers.1.1.0.bias
layers.1.1.1.weight
layers.1.1.3.weight
layers.2.0.norm_media.weight
layers.2.0.norm_media.bias
layers.2.0.norm_latents.weight
layers.2.0.norm_latents.bias
layers.2.0.to_q.weight
layers.2.0.to_kv.weight
layers.2.0.to_out.weight
layers.2.1.0.weight
layers.2.1.0.bias
layers.2.1.1.weight
layers.2.1.3.weight
layers.3.0.norm_media.weight
layers.3.0.norm_media.bias
layers.3.0.norm_latents.weight
layers.3.0.norm_latents.bias
layers.3.0.to_q.weight
layers.3.0.to_kv.weig

In [12]:
to_q = projector['layers.0.0.to_q.weight']
to_kv= projector['layers.0.0.to_kv.weight']
to_o = projector['layers.0.0.to_out.weight']

In [13]:
to_q.shape, to_kv.shape, to_o.shape

(torch.Size([1536, 1152]), torch.Size([3072, 1152]), torch.Size([1152, 1536]))

In [14]:
import re
re.sub("attn.in_proj_", "attn.q.", 'attn.in_proj_weight')

'attn.q.weight'